In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
products_df = spark.read.format('parquet').load('abfss://bronze@azadlsgen7428.dfs.core.windows.net/products')

In [0]:
products_df.display()

In [0]:
products_df.createOrReplaceTempView('products')

In [0]:
spark.sql("""
SELECT * FROM products
""").display()

In [0]:
%sql

create or replace function `ete-project`.bronze.discount_func(p_price double)
returns double
language sql
return p_price*0.90

In [0]:
%sql

select product_id,price,`ete-project`.bronze.discount_func(price) as discounted_price from products;

In [0]:
df = products_df.withColumn('discounted_price',expr('`ete-project` .bronze.discount_func(price)'))

In [0]:
df.display()

In [0]:
%sql
create or replace function `ete-project`.bronze.upper_funcs(p_brand string)
returns string
language python
as 
$$
  return p_brand.upper()
$$

In [0]:
%sql
select product_id,brand,`ete-project`.bronze.upper_funcs(brand) as upper_brand from products

In [0]:
df.write.format('delta').mode('overwrite').option('path','abfss://silver@azadlsgen7428.dfs.core.windows.net/products').save()

In [0]:
%sql

create table if not exists `ete-project`.silver.products
using delta
location 'abfss://silver@azadlsgen7428.dfs.core.windows.net/products'